In [1]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

/Users/duqiming2004/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Toy Example (dim = 1)
* $X \sim \mathcal{N}(0,1)$ r.v that we could simulate
* $S:\mathbb{R} \to \mathbb{R}$ score function / blackbox i.e. we could simulate $S(X)$, but we don't know the form of $S$. here we take $S(X) = |X| $
* Goal : estimate $p = \mathbb{P}(S(X)>q) < 10^{-6}$ ( q = 5 rare event)

In [2]:
from numba import autojit
@autojit
def S(X):
    '''score function which is a black box'''
    return np.abs(X)


### remark
(cf. Sequential Monte Carlo for Rare Event Estimation(F.Cérou, P.Del Moral, T.Furon, A.Guyader)):
http://www.lsta.lab.upmc.fr/modules/resources/download/labsta/Pages/Guyader/cdfg.pdf

## Fixed-levels Algorithm:

#### Parameters:

N: the number of particles

$\{L_0,...,L_n\}$: the sequence of levels, where $L_0 = -\infty$

#### Initialization

Draw an i.i.d. N-sample $(X_0^j)_{1\leq j\leq N}$ of law $\mu$

#### Iterations

*for k = 0 to n-1:*

Let $I_k  = \{j : X_k^j \in A_{k+1}\}$ where $A_{k+1} = \{x \in \mathbb{R}^d : S(x) > L_k\}$

Let $\hat p_k = \frac{\#|T_k|}{N}$

if $j \in I_k$, let $\tilde X_{k+1}^j = X_k^j$

if $j \notin I_k$, let $\tilde X_{k+1}^j$ be a copy of $X_k^l$ where $l$ is chosen randomly in $I_k$ with uniform
probabilities.



############# *Question* ################


The multinominal method of choosing $\tilde X_{k+1}^j$ becomes expensive when the particle number increase. At the same time, there would be a problem when $I_k$ is empty, which would be possible when the level becomes large (i.e the $A_{k+1}$ is rare), a possible solution is to regenerate an $X_{new} \sim \mathcal{L}( X | S(X)>L_{k+1})$ with rejection sampling(this could ensure that the $I_k$ is not empty). However, this method end up with an analogue version of classic monte-carlo procedure.



#### Solution:

We will use a trick to simplify the multinominal: we get new copie of $\tilde X_{k+1}^j$ by doing the permutation to $I_k$, i.e. we will use a deterministic method.

This procedure is far more effecient than the generation mentioned above. However, to ensure that $I_k$ is not empty for every level k, we should draw a big number of particles. 
########################################


*for j from 1 to N:*

Draw a new particle $\hat X_{k+1}^j \sim K(\tilde X_{k+1}^j,\cdot )$

if $\hat X_{k+1}^j \in A_{k+1}$, let $X_{k+1}^j = \hat X_{k+1}^j$, else let $X_{k+1}^j = \tilde X_{k+1}^j$

(that is to say, with the condition that $X \in A_{k+1}$, we will only accept the transition in $A_{k+1}$ )

#### Output

Estimate the proba of the rare events by $\hat p = \prod_{k = 0}^{n - 1} \hat p_k $

In [3]:
from scipy.stats import norm

# sequence of levels

q_test = 4

p = (1-norm.cdf(q_test))*2

###idealized situation
p_0 = 0.5 #success rate
n_0 = int(np.log(p)/np.log(p_0))
r = p/(p_0**n_0)

print "p_0 = ", p_0, '\t n_0 =',n_0,"\t r = ",r

L = [-np.Inf]
for k in range(1,n_0,1):
    L = np.append(L, norm.ppf(1-p_0**k/2))
L_ideal = np.append(L, q_test)
num_lev = len(L_ideal)

##var_relative
sigma_theoric = np.sqrt(n_0*(1-p_0)/p_0 + (1-r)/r)
print "sequence of levels: ", L_ideal
print "num_lev: ",num_lev
print "level interested, L = ",q_test
# real value of p
print "real value of p:" ,p
print "theoric relative variance: ", sigma_theoric

p_0 =  0.5 	 n_0 = 13 	 r =  0.518901626194
sequence of levels:  [       -inf  0.67448975  1.15034938  1.53412054  1.86273187  2.15387469
  2.41755902  2.66006747  2.88563491  3.09726908  3.29719335  3.4871041
  3.66832929  4.        ]
num_lev:  14
level interested, L =  4
real value of p: 6.33424836662e-05
theoric relative variance:  3.73190937117


In [ ]:
# (1-norm.cdf(3.66832929))*2*r
p_hat

In [4]:
#tuning parameter 
sigma_1 = 1
#std_tuning = np.sqrt(sigma_1**2)/(1+sigma_1**2)
std_tuning = np.float(sigma_1**2)/(1+sigma_1**2)
c = np.sqrt(1+sigma_1**2)
L = L_ideal


print "std_tuning: ",std_tuning
print "levels: ", L


###Estimation of p

N = 10000 # number of samples
X= np.random.normal(0,1,N) 
p_hat = []

for k in range(num_lev-1):
    I = []
    X_tilde = np.zeros(N)
    print "k = ", k 
    
    
###### construction of I_k 
    for j in range(N):
        if S(X[j])>L[k+1]:
            I = np.append(I,X[j])  
    l = len(I)
    p_hat = np.append(p_hat, l/np.float(N))
    #print "estimation of p_k" ,p_hat[k]
######

    X_tilde[0:l] = I
    I = np.random.permutation(I)
    for i in range(l,N,1):
        X_tilde[i] = I[i%l]
    
    for j in range(N):            
        X_iter = np.random.normal(X_tilde[j]/c,std_tuning,1)
        if S(X_iter)>L[k+1]:
            X[j] = X_iter
            
    #print "size of I_k: ", l
    #print"\t"
    
    

var_rel = (p - np.prod(p_hat))/p*np.sqrt(N)
print "real value of p:" ,p
print "estimation of p: ", np.prod(p_hat)
print "relative variation: ", var_rel
print "N: ",N






std_tuning:  0.5
levels:  [       -inf  0.67448975  1.15034938  1.53412054  1.86273187  2.15387469
  2.41755902  2.66006747  2.88563491  3.09726908  3.29719335  3.4871041
  3.66832929  4.        ]
k =  0
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
real value of p: 6.33424836662e-05
estimation of p:  5.43891736941e-13
relative variation:  99.9999991413
N:  10000


### remark
(cf. Sequential Monte Carlo for Rare Event Estimation(F.Cérou, P.Del Moral, T.Furon, A.Guyader)):
http://www.lsta.lab.upmc.fr/modules/resources/download/labsta/Pages/Guyader/cdfg.pdf


## Adaptive Multilevel Splitting    

#### parameter:

$N$:  the number of particles

$N_0$: the number of succeeding particles

$p_0 = \frac{N_0}{N}$ : the success rate

$L$: the level we want to estimate

#### Initialization

Draw an i.i.d. N-sample $(X_0^j)_{1\leq j\leq N}$ of law $\mu$

Compute the $\hat L_1$, the $(1-p_0)$ quantile of $(S(X_0^j))_{1\leq j \leq N}$

k = 1, index of level

#### Iterations

while $\hat L_k < L$ do:

Starting from an i.i.d. $p_0 N$-sample with law $\mathcal{L}(X|S(X) > L_k)$,
draw an i.i.d. $N$-sample $(X_k^j)_{1 \leq j \leq N}$ with the same law.

################# *remark* ##################

Here we use the same trick as in fixed-level splitting, i.e. to use a permutation to draw an an i.i.d. $(1-p_0)N$-sample $(X_k^j)_{1 \leq j \leq N}$ with the same law

############################################

Compute the $\hat L_{k+1}$, the $(1-p_0)$ quantile of $(X_k^j)_{1 \leq j \leq N}$

k = k + 1

end while

Calculate $N_L = \#\{j : S(X_{k-1}^j) \geq L\}$

#### Output

Estimate the probability of the rare event by $\hat p = \frac{N_L}{N}p_0^{k-1}$.

#### remark

* The adaptive version is biased 

$$bias \sim O(\frac1N)$$

* The bias is non-negative, so the estimation is always a little bit overvalued





In [11]:
q_test = 5
from scipy.stats import norm
p = (1-norm.cdf(q_test))*2
print "real value of p:" ,p




p_0 = 0.75 # prescribed success rate
N = 50000 #size of sample

# calculate the empirical quantile of X
#@autojit
def L_empirical(X,alpha, N):
    
    #N = len(X)
    #return np.percentile(np.sort(S(X)),(1-alpha)*100.,interpolation="lower")
    return np.sort(S(X))[np.int((1-alpha)*N)]

###Estimation of p

## To ensure that L_k != empty
X = np.random.normal(0,1,N)
while(np.sum((S(X)>q_test)) == 0):
    X = np.random.normal(0,1,N)

L = np.array([-np.Inf,L_empirical(X ,p_0 ,N )])
k = 1

while(L[k]<q_test):
    print "\t"
    print "k = ",k
    print 'current level: ', L[k]
    I = []
    
 

    for i in range(N):
        if S(X[i])>L[k]:
            I = np.append(I, X[i])
    l = len(I)
    print "size of I_k: ", l
       
    X[0:l] = I
    
########## permutation trick to replace multinominal distribution
    I = np.random.permutation(I)
    for i in range(l,N,1):
        X[i] = I[i%l]
    
            
# rejection sampling = naive m.c.           
#             while(S(X_tilde)<=L[k]):
#                 X_tilde = np.random.normal(0,1,1)
            

#            X_new = np.append(X_new, X_tilde)         
    
    L = np.append(L, L_empirical(X,p_0 ,N))
    k += 1

print "final k = ",k
    
N_L = np.sum((S(X)>q_test))
p_hat = N_L/float(N)*p_0**(k-1)
L_adapted = L[0:-1]
L_adapted = np.append(L_adapted, q_test)

print "real value of p:" ,p
print "estimation of p: ", p_hat
print "relative variation: ",np.abs((p - p_hat))/p*np.sqrt(N)
print "N: ",N



real value of p: 5.73303143847e-07
	
k =  1
current level:  0.315792917037
size of I_k:  37499
	
k =  2
current level:  0.574631450406
size of I_k:  37499
	
k =  3
current level:  0.799863580496
size of I_k:  37498
	
k =  4
current level:  1.0048239885
size of I_k:  37498
	
k =  5
current level:  1.18350584757
size of I_k:  37496
	
k =  6
current level:  1.34759376411
size of I_k:  37498
	
k =  7
current level:  1.50429095836
size of I_k:  37497
	
k =  8
current level:  1.64398678897
size of I_k:  37486
	
k =  9
current level:  1.78520831734
size of I_k:  37473
	
k =  10
current level:  1.91191289349
size of I_k:  37472
	
k =  11
current level:  2.03953065117
size of I_k:  37496
	
k =  12
current level:  2.15440841088
size of I_k:  37476
	
k =  13
current level:  2.26519863098
size of I_k:  37430
	
k =  14
current level:  2.37165248209
size of I_k:  37426
	
k =  15
current level:  2.48618874771
size of I_k:  37302
	
k =  16
current level:  2.5881412239
size of I_k:  37462
	
k =  17
cur

In [ ]:
L_adapted = L[0:-1]
L_adapted = np.append(L_adapted,q_test)
print "levels (adapted version): ", L_adapted

## Fluctuation Analysis

In this section we will do some analysis of the comportment of $\frac{\hat p - p}{p}$. In the idealized situation, by the thm of CTL, it converges to a normal law and we are interested in estimating the variance of this normal distribution.

In [ ]:
def fixed_sim(N,sigma_1 = 1, L = L_ideal):
    #tuning parameter 
    
    std_tuning = np.sqrt(sigma_1**2)/(1+sigma_1**2)
    c = np.sqrt(1+sigma_1**2)

    #print "std_tuning: ",std_tuning
    #print "levels: ", L


    ###Estimation of p

    X = np.random.normal(0,1,N)
    while(np.sum((S(X)>q_test)) == 0):
        X = np.random.normal(0,1,N)

 
    p_hat = []
    num_lev = len(L)
    for k in range(num_lev-1):
        I = []
        X_tilde = np.zeros(N)
        #print "k = ", k 


    ###### construction of I_k 
        for j in range(N):
            if S(X[j])>=L[k+1]:
                I = np.append(I,X[j])  
        l = len(I)
        p_hat = np.append(p_hat, l/float(N))
        #print "estimation of p_k" ,p_hat[k]
    ######

        X_tilde[0:l] = I
        I = np.random.permutation(I)
        for i in range(l,N,1):
            X_tilde[i] = I[i%l]
        #the shaker
        for j in range(N):            
            X_iter = np.random.normal(X_tilde[j]/c,std_tuning,1)
            if S(X_iter)>L[k+1]:
                X[j] = X_iter

        #print "size of I_k: ", l
        #print"\t"



    var_rel = (p - np.prod(p_hat))/p*np.sqrt(N)
    #print "real value of p:" ,p
    #print "estimation of p: ", np.prod(p_hat)
    #print "relative variation: ", var_rel
    #print "N: ",N
    return np.prod(p_hat),var_rel




In [12]:
def adapted_sim(N, q_test = 4, p_0 = 0.75):

    p = (1-norm.cdf(q_test))*2
#print "real value of p:" ,p




     # prescribed success rate


    # calculate the empirical quantile of X
    #@autojit

    ###Estimation of p

    ## To ensure that L_k != empty
    X = np.random.normal(0,1,N)
    while(np.sum((S(X)>q_test)) == 0):
        X = np.random.normal(0,1,N)

    L = np.array([-np.Inf,L_empirical(X, p_0, N)])
    k = 1

    while(L[k]<q_test):
        #print "\t"
        #print "k = ",k
        #print 'current level: ', L[k]
        I = []



        for i in range(N):
            if S(X[i])>L[k]:
                I = np.append(I, X[i])
        l = len(I)
        #print "size of I_k: ", l

        X[0:l] = I

    ########## permutation trick to replace multinominal distribution
        I = np.random.permutation(I)
        for i in range(l,N,1):
            X[i] = I[i%l]


    # rejection sampling = naive m.c.           
    #             while(S(X_tilde)<=L[k]):
    #                 X_tilde = np.random.normal(0,1,1)


    #            X_new = np.append(X_new, X_tilde)         

        L = np.append(L, L_empirical(X, p_0, N))
        k += 1

    #print "final k = ",k

    N_L = np.sum((S(X)>q_test))
    p_hat = N_L/float(N)*p_0**(k-1)
    L_adapted = L[0:-1]
    L_adapted = np.append(L_adapted, q_test)
    
    return p_hat, np.sqrt(N) * (p-p_hat)/p

#     print "real value of p:" ,p
#     print "estimation of p: ", p_hat
#     print "relative variation: ",np.abs((p - p_hat))/p*np.sqrt(N)
#     print "N: ",N




In [ ]:
# n_sim = 100
# #N = 100, 1000, 10000, 100000
# sim_N =[[] for i in range(4)]
# for n in range(4):
#     for i in range(n_sim):
#         sim_N[n] = fixed_sim(N = 10*10**n)

In [ ]:
#from numba import autojit, prange
n_sim = 1000
sim_5000 = [[] for i in range(n_sim)]
#@autojit
def parallel_sim():
    for i in range(n_sim):
        sim_5000[i] = adapted_sim(1000,3)
parallel_sim ()    
sim_5000

In [ ]:
plt.figure(figsize = [15,5])
plt.hist(np.array(sim_5000)[:,1],bins = 50)